# DAOFind, Peak Finding and Statistics Computation

Study ways to improve DAOFind, common values for statistics and peak finding. Comparing ASTROPOP implementation with Stetson (python port by D. Jones in [PythonPhot](https://github.com/djones1040/PythonPhot/)) and Astropy affiliated [Photutils](https://github.com/astropy/photutils).

In [ ]:
import sys
sys.path.append('.')
import numpy as np
from matplotlib import pyplot as plt
%matplotlib notebook
from scipy.ndimage.filters import convolve

from library.gen_image import gen_image

## Peak Finding

We will compare 3 algorithms here:
- Stetson DAOFind, that seeks local maximuns inside a box
- Photutils `peak_find`, used in Photutils source detection
- Scikit-image `peak_local_max`, that uses local max filter

All these algorithms will be applyed in the gausian-kernel convolved image.

### Using simulated stars

For the first peak fingind test, lets use a image with some strange sources. 3 round, 3 very alonged and 3 sharp cosmicray-like.

In [ ]:
# Image: # 3x3 "stars"
# first row just scraps, second row elipses, third row different fwhm
size = (512, 512)
posx = [128, 256, 384]*3
posy = np.ravel([np.arange(90, 120, step=10),
                 np.arange(240, 270, step=10),
                 np.arange(360, 390, step=10)])
flux = (0, 0, 0, 80000, 80000, 80000, 50000, 60000, 80000)
sigma_x = (1, 1, 1, 2, 10, 2, 1, 2, 4)
sigma_y = (1, 1, 1, 10, 2, 10, 1, 2, 4)
fwhm = 5
theta = (0, 0, 0, 0, 0, 45, 0, 0, 0)
sky = 800
rdnoise = 20  # very low noise
min_snr = 10

im = gen_image(size, posx, posy, flux, sky, rdnoise,
               model='gaussian', sigma=(sigma_x, sigma_y),
               theta=theta)

# crap sources
im[posy[0]:posy[0]+2, posx[0]:posx[0]+2] = 15000
im[posy[1]-5:posy[1]+5, posx[1]] = 15000
im[posy[1], posx[1]-5:posx[1]+5] = 15000
im[posy[2]-3:posy[2]+3, posx[2]-3:posx[2]+3] += 25000
im[posy[2], posx[2]] += 25000

fig, ax = plt.subplots(1, 2, figsize=(10, 6), sharex=True, sharey=True)
ax[0].imshow(im, origin='lower', vmax=12000)
ax[0].set_title('Original')

from library.stetson_daofind import stetson_image_params, stetson_kernels
image, hmin, n_x, n_y, radius, nhalf, nbox, _, sigsq = stetson_image_params(fwhm, min_snr, rdnoise, im, sky)
mask, g, pixels, c, c1 = stetson_kernels(radius, nhalf, nbox, sigsq)
h = convolve(image, c)
ax[1].imshow(h, origin='lower')
ax[1].set_title('Convolved')

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10, 4))

# Stetson daofind
from library.stetson_daofind import stetson_find_peaks
print('stetson')
%timeit -n1 stetson_find_peaks(h, hmin, mask, pixels, nhalf, n_x, n_y)
ix, iy, ngood = stetson_find_peaks(h, hmin, mask, pixels, nhalf, n_x, n_y)
ax[0].imshow(im, origin='lower', vmin=800, vmax=3200)
ax[0].plot(ix, iy, 'r.', alpha=0.5)
ax[0].set_title(f'Stetson DAOFind\n{len(ix)} sources')

# Using photutils default
from photutils.detection import find_peaks
print('photutils')
%timeit -n1 find_peaks(h, hmin)
peaks = find_peaks(h, hmin)
ax[1].imshow(im, origin='lower', vmin=800, vmax=3200)
ax[1].plot(peaks['x_peak'], peaks['y_peak'], 'r.', alpha=0.5)
ax[1].set_title(f'Photutils find_peaks\n{len(peaks)} sources')

# using scikit-image
from skimage.feature import peak_local_max
print('skimage')
%timeit -n1 peak_local_max(h, min_distance=int(fwhm), threshold_rel=hmin/np.max(h))
peaks = peak_local_max(h, min_distance=int(fwhm), threshold_rel=hmin/np.max(h))
ax[2].imshow(im, origin='lower', vmin=800, vmax=3200)
ax[2].plot(peaks[:, 1], peaks[:, 0], 'r.')
ax[2].set_title(f'Skimage peak_local_max\n{len(peaks)} sources')

plt.tight_layout()
plt.show()

### Conclusions on simulated image:
- Both Stetson and Photutils found spurious peaks, specially in the hard edge sources. Photutils found more of them.
- Scikit-image needed some tunning in the min_distance and threshold_rel parameters but these combinations seems to work well:
  - `threshold_rel=hmin/np.max(h)`
  - `min_distance=int(fwhm)`

### Real image

Lets try a real image from Photutils dataset. Again, applying the peak finding in the convolved image. This image has saturated stars, that appear with dark center on convolved image.

In [ ]:
from astropy.stats import sigma_clipped_stats
from photutils.datasets import load_star_image
hdu = load_star_image()
print(np.shape(hdu.data))
im = hdu.data[600:1001, 600:1001].astype('f8')
fwhm = 5
bkg = np.median(im)
rms = np.std(im)
snr = 10

fig, ax = plt.subplots(1, 2, figsize=(10, 6))
ax[0].imshow(im, origin='lower', vmax=12000)
ax[0].set_title('Original')

from library.stetson_daofind import stetson_image_params, stetson_kernels
image, hmin, n_x, n_y, radius, nhalf, nbox, _, sigsq = stetson_image_params(fwhm, min_snr, rms, im, bkg)
mask, g, pixels, c, c1 = stetson_kernels(radius, nhalf, nbox, sigsq)
h = convolve(image, c)
ax[1].imshow(h, origin='lower')
ax[1].set_title('Convolved')

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10, 4), sharex=True, sharey=True)

# Stetson daofind
from library.stetson_daofind import stetson_find_peaks
print('stetson')
%timeit -n1 stetson_find_peaks(h, hmin, mask, pixels, nhalf, n_x, n_y)
ix, iy, ngood = stetson_find_peaks(h, hmin, mask, pixels, nhalf, n_x, n_y)
ax[0].imshow(im, origin='lower')
ax[0].plot(ix, iy, 'r.', alpha=0.5)
ax[0].set_title(f'Stetson DAOFind\n{len(ix)} sources')

# Using photutils default
from photutils.detection import find_peaks
print('photutils')
%timeit -n1 find_peaks(h, hmin)
peaks = find_peaks(h, hmin)
ax[1].imshow(im, origin='lower')
ax[1].plot(peaks['x_peak'], peaks['y_peak'], 'r.', alpha=0.5)
ax[1].set_title(f'Photutils find_peaks\n{len(peaks)} sources')

# using scikit-image
from skimage.feature import peak_local_max
print('skimage')
%timeit -n1 peak_local_max(h, min_distance=int(fwhm), threshold_rel=hmin/np.max(h))
peaks = peak_local_max(h, min_distance=int(fwhm), threshold_rel=hmin/np.max(h))
ax[2].imshow(im, origin='lower')
ax[2].plot(peaks[:, 1], peaks[:, 0], 'r.', alpha=0.5)
ax[2].set_title(f'Skimage peak_local_max\n{len(peaks)} sources')

plt.tight_layout()
plt.show()

### Conclusions on real image

Here, the results are almost identical, but skimage lost some weak sources near brighter peaks. Adjusting the `min_distance` to `fwhm/1.5` this problem is solved. However, using this value in the simulated image make skimage find exactly the same sources as Stetson algorith.

All the algorithms ignored saturated stars, due to the valleys in the center of the convolved image.